In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#export
from nb_004c import *

# Carvana

## Setup

(See final section of notebook for one-time data processing steps.)

In [3]:
PATH = Path('data/carvana')
PATH_PNG = PATH/'train_masks_png'
PATH_X = PATH/'train-128'
PATH_Y = PATH/'train_masks-128'

In [4]:
img_f = next(PATH_X.iterdir())
x = open_image(img_f)
show_image(x)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data\\carvana\\train-128'

In [ ]:
def get_y_fn(x_fn): return PATH_Y/f'{x_fn.name[:-4]}_mask.png'

In [ ]:
#export
def pil2tensor(image, as_mask=False):
    arr = torch.ByteTensor(torch.ByteStorage.from_buffer(image.tobytes()))
    arr = arr.view(image.size[1], image.size[0], -1)
    arr = arr.permute(2,0,1).float()
    return arr if as_mask else arr.div_(255)

def open_image(fn, as_mask=False):
    x = PIL.Image.open(fn)
    if not as_mask: x = x.convert('RGB')
    return pil2tensor(x, as_mask=as_mask)

def image2np(image):
    res = image.cpu().permute(1,2,0).numpy()
    return res[...,0] if res.shape[2]==1 else res

def show_image(img, ax=None, figsize=(3,3), hide_axis=True, cmap='binary'):
    if ax is None: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(image2np(img), cmap=cmap)
    if hide_axis: ax.axis('off')

In [ ]:
img_y_f = get_y_fn(img_f)
y = open_image(img_y_f, as_mask=True)
show_image(y)

In [ ]:
tfms = [flip_lr_tfm(p=0.5),
        rotate_tfm(degrees=(-20,20.), p=0.75),
        zoom_tfm(scale=(0.5,2), p=0.75),
        contrast_tfm(scale=(0.6,1.4)),
        brightness_tfm(change=(0.3,0.7))
]

In [ ]:
tfms

In [ ]:
_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes: show_image(apply_tfms(tfms)(x), ax)

## xy transforms

In [ ]:
#export
def _apply_tfm_func1(pixel_func,lighting_func,affine_func,start_func, x, **kwargs):
    if not np.any([pixel_func,lighting_func,affine_func,start_func]): return x
    x = x.clone()
    if start_func is not None:  x = start_func(x)
    if affine_func is not None: x = affine_func(x, **kwargs)
    if lighting_func is not None: x = lighting_func(x)
    if pixel_func is not None: x = pixel_func(x)
    return x

def _apply_tfm_funcs(pixel_func,lighting_func,affine_func,start_func,
                     x, y=None, segmentation=False, **kwargs):
    x = _apply_tfm_func1(pixel_func,lighting_func,affine_func,start_func, x, **kwargs)
    if y is None: return x
    
    if segmentation: lighting_func=None
    y = _apply_tfm_func1(pixel_func, lighting_func,affine_func,start_func, y,
                         mode='nearest' if segmentation else 'bilinear', **kwargs)
    return x,y

import nb_003a
nb_003a._apply_tfm_funcs = _apply_tfm_funcs

In [ ]:
_,axes = plt.subplots(2,4, figsize=(12,6))
for i in range(4):
    imgx,imgy = apply_tfms(tfms)(x,y, segmentation=True)
    show_image(imgx, axes[0][i])
    show_image(imgy, axes[1][i])

In [ ]:
_,axes = plt.subplots(2,4, figsize=(12,6))
for i in range(4):
    imgx,imgy = apply_tfms(tfms)(x,y, segmentation=False)
    show_image(imgx, axes[0][i])
    show_image(imgy, axes[1][i], cmap='viridis')

## Dataset

In [ ]:
#export
@dataclass
class MatchedFilesDataset(Dataset):
    x_fns:List[Path]; y_fns:List[Path]
    def __post_init__(self): assert len(self.x_fns)==len(self.y_fns)
    def __repr__(self): return f'{type(self).__name__} of len {len(self.x_fns)}'
    def __len__(self): return len(self.x_fns)
    def __getitem__(self, i): return open_image(self.x_fns[i]), open_image(self.y_fns[i])

In [ ]:
x_fns = [o for o in PATH_X.iterdir() if o.is_file()]
y_fns = [get_y_fn(o) for o in x_fns]
train_ds = MatchedFilesDataset(x_fns, y_fns)
train_ds

In [ ]:
_,axes = plt.subplots(2,4, figsize=(12,6))
for i in range(4):
    imgx,imgy = train_ds[i]
    show_image(imgx, axes[0][i])
    show_image(imgy, axes[1][i])

In [ ]:
#export
default_mean,default_std = map(tensor, ([0.5]*3, [0.5]*3))

class TfmDataset(Dataset):
    def __init__(self, ds:Dataset, tfms:Collection[Callable]=None,
                 do_tfm_y=False, smooth_y=True, **kwargs):
        self.ds,self.tfms,self.do_tfm_y,self.smooth_y,self.kwargs = ds,tfms,do_tfm_y,smooth_y,kwargs

    def __len__(self): return len(self.ds)

    def __getitem__(self,idx):
        x,y = self.ds[idx]
        if self.tfms is not None:
            tfm = apply_tfms(self.tfms)
            if self.do_tfm_y: x,y = tfm(x,y, self.smooth_y, **self.kwargs)
            else:             x   = tfm(x, **self.kwargs)
        return x,y

In [ ]:
train_tds = TfmDataset(train_ds, tfms, do_tfm_y=True, smooth_y=False)

In [ ]:
_,axes = plt.subplots(4,4, figsize=(12,12))
for i in range(4):
    imgx,imgy   = train_ds[i]
    imgx2,imgy2 = train_tds[i]
    show_image(imgx,  axes[0][i])
    show_image(imgx2, axes[1][i])
    show_image(imgy,  axes[2][i])
    show_image(imgy2, axes[3][i])

## Create model

In [ ]:
default_norm = normalize_tfm(mean=default_mean, std=default_std)
default_denorm = partial(denormalize, mean=default_mean, std=default_std)

In [ ]:
#export
class Transforms():
    def __init__(self, tfms:Collection[Callable]=None, **kwargs):
        self.tfms,self.kwargs = tfms,kwargs
        
    def __call__(self, x):
        return x if self.tfms is None else apply_tfms(self.tfms)(x, **self.kwargs)

@dataclass
class TfmDataset(Dataset):
    ds:Dataset; x_tfms:Transforms=None; y_tfms:Transforms=None

    def __len__(self): return len(self.ds)

    def __getitem__(self,idx):
        x,y = self.ds[idx]
        return self.x_tfms(x),self.y_tfms(y)

In [ ]:
#export
class Darknet(nn.Module):
    def make_group_layer(self, ch_in, num_blocks, stride=1):
        return [conv_layer(ch_in, ch_in*2,stride=stride)
               ] + [(ResLayer(ch_in*2)) for i in range(num_blocks)]

    def __init__(self, num_blocks, num_classes, nf=32, custom_head=None):
        super().__init__()
        layers = [conv_layer(3, nf, ks=3, stride=1)]
        for i,nb in enumerate(num_blocks):
            layers += self.make_group_layer(nf, nb, stride=2-(i==1))
            nf *= 2
        layers += [nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(nf, num_classes)
                  ] if custom_head is None else custom_head
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x): return self.layers(x)

# Convert and resize data

In [ ]:
def convert_img(fn): Image.open(fn).save(PATH_PNG/f'{fn.name[:-4]}.png')

def resize_img(fn, dirname):
    Image.open(fn).resize((128,128)).save((fn.parent.parent)/dirname/fn.name)

def do_conversion():
    PATH_PNG.mkdir(exist_ok=True)
    PATH_X.mkdir(exist_ok=True)
    PATH_Y.mkdir(exist_ok=True)

    files = list((PATH/'train_masks').iterdir())
    with ThreadPoolExecutor(8) as e: e.map(convert_img, files)

    files = list(PATH_PNG).iterdir())
    with ThreadPoolExecutor(8) as e: e.map(partial(resize_img, dirname='train_masks-128'), files)

    files = list((PATH/'train').iterdir())
    with ThreadPoolExecutor(8) as e: e.map(partial(resize_img, dirname='train-128'), files)

In [ ]:
# do_conversion()